## Grabing Public Hotel Occupancy Tax Data, then storing it into a database, crossreferencing if data is repeating

#### Prerequisites:
##### requirements for mysql-python communication:
* pip install mysqlclient
* pip install mysql-connector-python 
* if recieving wheel error: pip install wheel

In [1]:
# Imports
import time
import sys
from zipfile import ZipFile
import pandas as pd
import pandas.io.sql as pdsql
import glob, os
import numpy as np
# Datetime for new column
import datetime
# Imports for mySQL
from sqlalchemy import create_engine, event, DateTime
from db_setup import mysql_user, mysql_password, db_name
import mysql.connector

#### File path defined

In [2]:
mydir = os.path.abspath('./HotelOccupancyTaxData')

In [3]:
mydir

'C:\\DataAnalyticsBootCamp\\WEEK_23 - Project 3\\Project_3_Potential_Marketing\\CRE_Marketing_Data\\HotelOccupancyTaxData'

## Defining headers for data

In [4]:
# Defining header for marketing data. Marketing data comes with no header
# Franchise tax permit
ftact_date_head = ['Taxpayer_Number',
                   'Taxpayer_Name',
                   'Taxpayer_Address',
                   'Taxpayer_City',
                   'Taxpayer_State',
                   'Taxpayer_Zip_Code',
                   'Taxpayer_County_Code',
                   'Taxpayer_Organizational_Type',
                   'Taxpayer_Phone_Number',
                   'Record_Type_Code',
                   'Responsibility_Beginning_Date',
                   'Secretary_of_State_File_Number',
                   'SOS_Charter_Date',
                   'SOS_Status_Date',
                   'Current_Exempt_Reason_Code',
                   'Agent_Name',
                   'Agent_Address',
                   'Agent_City',
                   'Agent_State',
                   'Agent_Zip_Code']
# Franchise tax permit date
ftact_head = ['Taxpayer_Number',
              'Taxpayer_Name',
              'Taxpayer_Address',
              'Taxpayer_City',
              'Taxpayer_State',
              'Taxpayer_Zip_Code',
              'Taxpayer_County_Code',
              'Taxpayer_Organizational_Type',
              'Taxpayer_Phone_Number',
              'Record_Type_Code',
              'Responsibility_Beginning_Date',
              'Responsibility_End_Date',
              'Responsibility_End_Reason_Code',
              'Secretary_of_State_File_Number',
              'SOS_Charter_Date',
              'SOS_Status_Date',
              'SOS_Status_Code',
              'Rigth_to_Tansact_Business_Code',
              'Current_Exempt_Reason_Code',
              'Exempt_Begin_Date',
              'NAICS_Code']

## Extract files from zipped folder

In [5]:
# extract all files
i = 0
for file in glob.glob(mydir + '/*.zip'):
    i += 1
    zip = ZipFile(file, 'r')
    print(f'Extracting file {i}')
    zip.extractall(mydir)
    zip.close()
    print('Done!')
    print(f"File {i}, extracted: {file}\n")
    time.sleep(1)
    os.remove(file)

Extracting file 1
Done!
File 1, extracted: C:\DataAnalyticsBootCamp\WEEK_23 - Project 3\Project_3_Potential_Marketing\CRE_Marketing_Data\HotelOccupancyTaxData\FTACT.zip

Extracting file 2
Done!
File 2, extracted: C:\DataAnalyticsBootCamp\WEEK_23 - Project 3\Project_3_Potential_Marketing\CRE_Marketing_Data\HotelOccupancyTaxData\PP_files.zip

Extracting file 3
Done!
File 3, extracted: C:\DataAnalyticsBootCamp\WEEK_23 - Project 3\Project_3_Potential_Marketing\CRE_Marketing_Data\HotelOccupancyTaxData\Real_building_land.zip

Extracting file 4
Done!
File 4, extracted: C:\DataAnalyticsBootCamp\WEEK_23 - Project 3\Project_3_Potential_Marketing\CRE_Marketing_Data\HotelOccupancyTaxData\STACT.zip



## Add csv files to a data frame ( fran and stp)

In [ ]:
# Searches for a csv file
df_fran = pd.DataFrame()

for file in glob.glob(mydir + '/*.csv'):
    if 'fran' in file:
        df = pd.read_csv(file, header=None, index_col=False, names=ftact_date_head, engine ='python')
        df_fran = df_fran.append(df)
        os.remove(file)
        print('Added the ' + file + " into the DF df_fran")
        print("deleted the file " + str(file))
    else:
        print('we do not know what to do with this file: ' + str(file))

# FRAN DF created

In [227]:
df_fran.head()

,Taxpayer_Number,Taxpayer_Name,Taxpayer_Address,Taxpayer_City,Taxpayer_State,Taxpayer_Zip_Code,Taxpayer_County_Code,Taxpayer_Organizational_Type,Taxpayer_Phone_Number,Record_Type_Code,Responsibility_Beginning_Date,Secretary_of_State_File_Number,SOS_Charter_Date,SOS_Status_Date,Current_Exempt_Reason_Code,Agent_Name,Agent_Address,Agent_City,Agent_State,Agent_Zip_Code
0,12020053802,"KAGER INDUSTRIES, LLC ...",3168 N STATE HIGHWAY 16,FREDERICKSBRG,TX,78624,86,CL,8309979391,U,20200101,0803491095,20200101,20200101,,ROBERT KAGER ...,3168 NORTH STATE HWY. 16,FREDERICKSBURG,TX,78624
1,13415672842,FLAMING RIVER INDUSTRIES INC ...,800 POERTNER DR,BEREA,OH,44017,0,CF,4408264488,X,20200101,,,,,...,,,,
2,13513654320,LA SALLE COMPANY INC ...,4315 RALPH JONES CT,SOUTH BEND,IN,46628,0,CF,5742717644,X,20200101,,,,,...,,,,
3,13636316849,LABSOURCE INC ...,1260 GARNET DR,NORTHLAKE,IL,60164,0,CF,6304135641,X,20200101,,,,,...,,,,
4,14610272230,RUSHING TRANSPORT LLC ...,7043 E CAPRICE AVE,BATON ROUGE,LA,70811,0,CI,2252688365,X,20191220,,,,,...,,,,


#### Adding the Taxpayer County Name and Record Type Name Column

In [228]:
# Taxpayer Organization Type:
df_fran.loc[(df_fran.Taxpayer_Organizational_Type == 'CF'),'Taxpayer_Organizational_Name']='Foreign Profit' #    CF - Foreign Profit
df_fran.loc[(df_fran.Taxpayer_Organizational_Type == 'CI'),'Taxpayer_Organizational_Name']='Limited Liability Company - Foreign'#   CI - Limited Liability Company - Foreign
df_fran.loc[(df_fran.Taxpayer_Organizational_Type == 'CL'),'Taxpayer_Organizational_Name']='Limited Liability Company - Texas' #   CL - Limited Liability Company - Texas
df_fran.loc[(df_fran.Taxpayer_Organizational_Type == 'CM'),'Taxpayer_Organizational_Name']='Foreign Non-Profit' #   CM - Foreign Non-Profit
df_fran.loc[(df_fran.Taxpayer_Organizational_Type == 'CN'),'Taxpayer_Organizational_Name']='Texas Non-Profit' #   CN - Texas Non-Profit
df_fran.loc[(df_fran.Taxpayer_Organizational_Type == 'CP'),'Taxpayer_Organizational_Name']='Professional' #   CP - Professional
df_fran.loc[(df_fran.Taxpayer_Organizational_Type == 'CR'),'Taxpayer_Organizational_Name']='Texas Insurance' #   CR - Texas Insurance
df_fran.loc[(df_fran.Taxpayer_Organizational_Type == 'CS'),'Taxpayer_Organizational_Name']='Foreign Insurance - OOS' #   CS - Foreign Insurance - OOS
df_fran.loc[(df_fran.Taxpayer_Organizational_Type == 'CT'),'Taxpayer_Organizational_Name']='Texas Profit' #   CT - Texas Profit
df_fran.loc[(df_fran.Taxpayer_Organizational_Type == 'CW'),'Taxpayer_Organizational_Name']='Texas Railroad Corporation' #   CW - Texas Railroad Corporation
df_fran.loc[(df_fran.Taxpayer_Organizational_Type == 'CX'),'Taxpayer_Organizational_Name']='Foreign Railroad Corporation - OOS' #   CX - Foreign Railroad Corporation - OOS
# Record Type Code:
df_fran.loc[(df_fran.Record_Type_Code == 'U'),'Record_Type_Name']='Secretary of State (SOS) File Number' #   U = Secretary of State (SOS) File Number
df_fran.loc[(df_fran.Record_Type_Code == 'V'),'Record_Type_Name']='SOS Certificate of Authority (COA) File Number' #   V = SOS Certificate of Authority (COA) File Number
df_fran.loc[(df_fran.Record_Type_Code == 'X'),'Record_Type_Name']='Comptroller Assigned File Number' #   X = Comptroller Assigned File Number
df_fran.head()

,Taxpayer_Number,Taxpayer_Name,Taxpayer_Address,Taxpayer_City,Taxpayer_State,Taxpayer_Zip_Code,Taxpayer_County_Code,Taxpayer_Organizational_Type,Taxpayer_Phone_Number,Record_Type_Code,...,SOS_Charter_Date,SOS_Status_Date,Current_Exempt_Reason_Code,Agent_Name,Agent_Address,Agent_City,Agent_State,Agent_Zip_Code,Taxpayer_Organizational_Name,Record_Type_Name
0,12020053802,"KAGER INDUSTRIES, LLC ...",3168 N STATE HIGHWAY 16,FREDERICKSBRG,TX,78624,86,CL,8309979391,U,...,20200101,20200101,,ROBERT KAGER ...,3168 NORTH STATE HWY. 16,FREDERICKSBURG,TX,78624,Limited Liability Company - Texas,Secretary of State (SOS) File Number
1,13415672842,FLAMING RIVER INDUSTRIES INC ...,800 POERTNER DR,BEREA,OH,44017,0,CF,4408264488,X,...,,,,...,,,,,Foreign Profit,Comptroller Assigned File Number
2,13513654320,LA SALLE COMPANY INC ...,4315 RALPH JONES CT,SOUTH BEND,IN,46628,0,CF,5742717644,X,...,,,,...,,,,,Foreign Profit,Comptroller Assigned File Number
3,13636316849,LABSOURCE INC ...,1260 GARNET DR,NORTHLAKE,IL,60164,0,CF,6304135641,X,...,,,,...,,,,,Foreign Profit,Comptroller Assigned File Number
4,14610272230,RUSHING TRANSPORT LLC ...,7043 E CAPRICE AVE,BATON ROUGE,LA,70811,0,CI,2252688365,X,...,,,,...,,,,,Limited Liability Company - Foreign,Comptroller Assigned File Number


#### Date format

In [229]:
# df_fran['SOS_Charter_Date'] = df_fran['SOS_Charter_Date'].str.strip()

df_fran['SOS_Charter_Date'] = df_fran['SOS_Charter_Date'].fillna(0)
df_fran['SOS_Status_Date'] = df_fran['SOS_Status_Date'].fillna(0)

# df_fran['SOS_Charter_Date'] = df_fran['SOS_Charter_Date'].astype(np.int64)
# df_fran['SOS_Status_Date'] = df_fran['SOS_Status_Date'].astype(np.int64)
df_fran['Responsibility_Beginning_Date'] = df_fran['Responsibility_Beginning_Date'].astype(np.int64)

df_fran['SOS_Charter_Date'] = pd.to_datetime(df_fran["SOS_Charter_Date"], format='%Y%m%d', errors='coerce')
df_fran['SOS_Status_Date'] = pd.to_datetime(df_fran["SOS_Status_Date"], format='%Y%m%d', errors='coerce')
df_fran['Responsibility_Beginning_Date'] = pd.to_datetime(df_fran["Responsibility_Beginning_Date"], format='%Y%m%d', errors='coerce')

df_fran['SOS_Charter_Date'] =df_fran['SOS_Charter_Date'].dt.normalize()
df_fran['SOS_Status_Date'] = df_fran['SOS_Status_Date'].dt.normalize()
df_fran['Responsibility_Beginning_Date'] = df_fran['Responsibility_Beginning_Date'].dt.normalize()

df_fran = df_fran[df_fran['Taxpayer_Zip_Code']!=0]

df_fran.head()

,Taxpayer_Number,Taxpayer_Name,Taxpayer_Address,Taxpayer_City,Taxpayer_State,Taxpayer_Zip_Code,Taxpayer_County_Code,Taxpayer_Organizational_Type,Taxpayer_Phone_Number,Record_Type_Code,...,SOS_Charter_Date,SOS_Status_Date,Current_Exempt_Reason_Code,Agent_Name,Agent_Address,Agent_City,Agent_State,Agent_Zip_Code,Taxpayer_Organizational_Name,Record_Type_Name
0,12020053802,"KAGER INDUSTRIES, LLC ...",3168 N STATE HIGHWAY 16,FREDERICKSBRG,TX,78624,86,CL,8309979391,U,...,2020-01-01,2020-01-01,,ROBERT KAGER ...,3168 NORTH STATE HWY. 16,FREDERICKSBURG,TX,78624,Limited Liability Company - Texas,Secretary of State (SOS) File Number
1,13415672842,FLAMING RIVER INDUSTRIES INC ...,800 POERTNER DR,BEREA,OH,44017,0,CF,4408264488,X,...,NaT,NaT,,...,,,,,Foreign Profit,Comptroller Assigned File Number
2,13513654320,LA SALLE COMPANY INC ...,4315 RALPH JONES CT,SOUTH BEND,IN,46628,0,CF,5742717644,X,...,NaT,NaT,,...,,,,,Foreign Profit,Comptroller Assigned File Number
3,13636316849,LABSOURCE INC ...,1260 GARNET DR,NORTHLAKE,IL,60164,0,CF,6304135641,X,...,NaT,NaT,,...,,,,,Foreign Profit,Comptroller Assigned File Number
4,14610272230,RUSHING TRANSPORT LLC ...,7043 E CAPRICE AVE,BATON ROUGE,LA,70811,0,CI,2252688365,X,...,NaT,NaT,,...,,,,,Limited Liability Company - Foreign,Comptroller Assigned File Number


#### Checking column count

In [230]:
df_fran.count()

Taxpayer_Number                   269021
Taxpayer_Name                     269021
Taxpayer_Address                  269021
Taxpayer_City                     269021
Taxpayer_State                    269021
Taxpayer_Zip_Code                 269021
Taxpayer_County_Code              269021
Taxpayer_Organizational_Type      269021
Taxpayer_Phone_Number             269021
Record_Type_Code                  269021
Responsibility_Beginning_Date     269021
Secretary_of_State_File_Number    269021
SOS_Charter_Date                  260148
SOS_Status_Date                   260148
Current_Exempt_Reason_Code        269021
Agent_Name                        269021
Agent_Address                     269021
Agent_City                        269021
Agent_State                       269021
Agent_Zip_Code                    269021
Taxpayer_Organizational_Name      262696
Record_Type_Name                  269021
dtype: int64

## Extracting textfile and storing into DF (FTOFFDIR, FTACT, STACT)

In [6]:
for file in glob.glob(mydir + '/*.txt'):
    if 'FTACT' in file:
        df_ftact = pd.read_fwf(file,
                               widths=[11, 50, 40, 20, 2, 5, 3, 2, 10, 1, 8, 8, 2, 10, 8, 8, 2, 1, 3, 8, 6],
                               header=None,
                               names=ftact_head, index_col=False, engine= 'python')  # FTOOB, FTACT
        df_ftact = df_ftact.append(df_ftact)
        os.remove(file)
        print('Added the ' + file + ' into df_ftact')
        print('deleted the file ' + str(file))
    else:
        os.remove(file)
        print('File not being used: ' + str(file))

File not being used: C:\DataAnalyticsBootCamp\WEEK_23 - Project 3\Project_3_Potential_Marketing\CRE_Marketing_Data\HotelOccupancyTaxData\building_other.txt
File not being used: C:\DataAnalyticsBootCamp\WEEK_23 - Project 3\Project_3_Potential_Marketing\CRE_Marketing_Data\HotelOccupancyTaxData\building_res.txt
File not being used: C:\DataAnalyticsBootCamp\WEEK_23 - Project 3\Project_3_Potential_Marketing\CRE_Marketing_Data\HotelOccupancyTaxData\exterior.txt
File not being used: C:\DataAnalyticsBootCamp\WEEK_23 - Project 3\Project_3_Potential_Marketing\CRE_Marketing_Data\HotelOccupancyTaxData\extra_features.txt
File not being used: C:\DataAnalyticsBootCamp\WEEK_23 - Project 3\Project_3_Potential_Marketing\CRE_Marketing_Data\HotelOccupancyTaxData\extra_features_detail1.txt
File not being used: C:\DataAnalyticsBootCamp\WEEK_23 - Project 3\Project_3_Potential_Marketing\CRE_Marketing_Data\HotelOccupancyTaxData\extra_features_detail2.txt
File not being used: C:\DataAnalyticsBootCamp\WEEK_23 - 

# FTACT DF created

In [57]:
df_ftact.head()

,Taxpayer_Number,Taxpayer_Name,Taxpayer_Address,Taxpayer_City,Taxpayer_State,Taxpayer_Zip_Code,Taxpayer_County_Code,Taxpayer_Organizational_Type,Taxpayer_Phone_Number,Record_Type_Code,...,Responsibility_End_Date,Responsibility_End_Reason_Code,Secretary_of_State_File_Number,SOS_Charter_Date,SOS_Status_Date,SOS_Status_Code,Rigth_to_Tansact_Business_Code,Current_Exempt_Reason_Code,Exempt_Begin_Date,NAICS_Code
0,10004813456,MARSHALL E HAMILTON MD PA.,PO BOX 38206 C/O STEVE WINN CPA,DALLAS,TX,75238.0,57,AP,9726442142,X,...,NaN,0,NaN,NaN,NaN,NaN,A,NaN,NaN,621111.0
1,10058125732,CAP INVESTMENT GROUP,1801 HINKLE DR STE 150,DENTON,TX,76201.0,61,PB,9408914223,X,...,NaN,0,NaN,NaN,NaN,NaN,A,NaN,NaN,551112.0
2,10082832105,"JDM RESOURCES, LTD",5105 BUCKLAND DR,MCKINNEY,TX,75070.0,43,PL,2145440124,X,...,NaN,0,NaN,NaN,NaN,NaN,A,NaN,NaN,525990.0
3,10100464006,CLARK INSURANCE,1945 CONGRESS ST STE A,PORTLAND,ME,4102.0,0,CF,2075232218,V,...,NaN,0,800878850.0,20071001.0,20071001.0,A,A,NaN,NaN,NaN
4,10100798122,"FIASCHETTI ENTERPRISES, LP",5610 5TH ST,KATY,TX,77493.0,101,PL,8329717661,U,...,NaN,0,800242075.0,20030826.0,20090520.0,R,A,NaN,NaN,NaN


#### Taxpayer_Organizational_Name and Record_Type_Name Column

In [58]:
# Taxpayer Organization Type:
df_ftact.loc[(df_ftact.Taxpayer_Organizational_Type == 'AB'),'Taxpayer_Organizational_Name']='Texas Business Association' #   AB – Texas Business Association
df_ftact.loc[(df_ftact.Taxpayer_Organizational_Type == 'AC'),'Taxpayer_Organizational_Name']='Foreign Business Association' #   AC – Foreign Business Association
df_ftact.loc[(df_ftact.Taxpayer_Organizational_Type == 'AF'),'Taxpayer_Organizational_Name']='Foreign Professional Association' #   AF – Foreign Professional Association
df_ftact.loc[(df_ftact.Taxpayer_Organizational_Type == 'AP'),'Taxpayer_Organizational_Name']='Texas Professional Association' #   AP – Texas Professional Association
df_ftact.loc[(df_ftact.Taxpayer_Organizational_Type == 'AR'),'Taxpayer_Organizational_Name']='Other Association' #   AR – Other Association
df_ftact.loc[(df_ftact.Taxpayer_Organizational_Type == 'CF'),'Taxpayer_Organizational_Name']='Foreign Profit' #   CF - Foreign Profit
df_ftact.loc[(df_ftact.Taxpayer_Organizational_Type == 'CI'),'Taxpayer_Organizational_Name']='Limited Liability Company - Foreign' #   CI - Limited Liability Company - Foreign
df_ftact.loc[(df_ftact.Taxpayer_Organizational_Type == 'CL'),'Taxpayer_Organizational_Name']='Limited Liability Company - Texas' #   CL - Limited Liability Company - Texas
df_ftact.loc[(df_ftact.Taxpayer_Organizational_Type == 'CM'),'Taxpayer_Organizational_Name']='Foreign Non-Profit' #   CM - Foreign Non-Profit
df_ftact.loc[(df_ftact.Taxpayer_Organizational_Type == 'CN'),'Taxpayer_Organizational_Name']='Texas Non-Profit' #   CN - Texas Non-Profit
df_ftact.loc[(df_ftact.Taxpayer_Organizational_Type == 'CP'),'Taxpayer_Organizational_Name']='Professional' #   CP - Professional
df_ftact.loc[(df_ftact.Taxpayer_Organizational_Type == 'CR'),'Taxpayer_Organizational_Name']='Texas Insurance' #   CR - Texas Insurance
df_ftact.loc[(df_ftact.Taxpayer_Organizational_Type == 'CS'),'Taxpayer_Organizational_Name']='Foreign Insurance - OOS' #   CS - Foreign Insurance - OOS
df_ftact.loc[(df_ftact.Taxpayer_Organizational_Type == 'CT'),'Taxpayer_Organizational_Name']='Texas Profit' #   CT - Texas Profit
df_ftact.loc[(df_ftact.Taxpayer_Organizational_Type == 'CU'),'Taxpayer_Organizational_Name']='Foreign Professional Corporation' #   CU – Foreign Professional Corporation
df_ftact.loc[(df_ftact.Taxpayer_Organizational_Type == 'CW'),'Taxpayer_Organizational_Name']='Texas Railroad Corporation' #   CW - Texas Railroad Corporation
df_ftact.loc[(df_ftact.Taxpayer_Organizational_Type == 'CX'),'Taxpayer_Organizational_Name']='Foreign Railroad Corporation - OOS' #   CX - Foreign Railroad Corporation – OOS
df_ftact.loc[(df_ftact.Taxpayer_Organizational_Type == 'HF'),'Taxpayer_Organizational_Name']='Foreign Holding Company' #   HF – Foreign Holding Company
df_ftact.loc[(df_ftact.Taxpayer_Organizational_Type == 'PB'),'Taxpayer_Organizational_Name']='Business General Partnership' #   PB – Business General Partnership
df_ftact.loc[(df_ftact.Taxpayer_Organizational_Type == 'PF'),'Taxpayer_Organizational_Name']='Foreign Limited Partnership' #   PF – Foreign Limited Partnership
df_ftact.loc[(df_ftact.Taxpayer_Organizational_Type == 'PI'),'Taxpayer_Organizational_Name']='Individual General Partnership' #   PI – Individual General Partnership
df_ftact.loc[(df_ftact.Taxpayer_Organizational_Type == 'PL'),'Taxpayer_Organizational_Name']='Texas Limited Partnership' #   PL – Texas Limited Partnership
df_ftact.loc[(df_ftact.Taxpayer_Organizational_Type == 'PV'),'Taxpayer_Organizational_Name']='Texas Joint Venture' #   PV – Texas Joint Venture
df_ftact.loc[(df_ftact.Taxpayer_Organizational_Type == 'PW'),'Taxpayer_Organizational_Name']='Foreign Joint Venture' #   PW – Foreign Joint Venture
df_ftact.loc[(df_ftact.Taxpayer_Organizational_Type == 'PX'),'Taxpayer_Organizational_Name']='Texas Limited Liability Partnership' #   PX – Texas Limited Liability Partnership
df_ftact.loc[(df_ftact.Taxpayer_Organizational_Type == 'PY'),'Taxpayer_Organizational_Name']='Foreign Limited Liability Partnerhsip' #   PY – Foreign Limited Liability Partnerhsip
df_ftact.loc[(df_ftact.Taxpayer_Organizational_Type == 'SF'),'Taxpayer_Organizational_Name']='Foreign Joint Stock Company' #   SF – Foreign Joint Stock Company
df_ftact.loc[(df_ftact.Taxpayer_Organizational_Type == 'ST'),'Taxpayer_Organizational_Name']='Texas Joint Stock Company' #   ST – Texas Joint Stock Company
df_ftact.loc[(df_ftact.Taxpayer_Organizational_Type == 'TF'),'Taxpayer_Organizational_Name']='Foreign Business Trust' #   TF – Foreign Business Trust
df_ftact.loc[(df_ftact.Taxpayer_Organizational_Type == 'TH'),'Taxpayer_Organizational_Name']='Texas Real Estate Investment Trust' #   TH – Texas Real Estate Investment Trust
df_ftact.loc[(df_ftact.Taxpayer_Organizational_Type == 'TI'),'Taxpayer_Organizational_Name']='Foreign Real Estate Investment Trust' #   TI – Foreign Real Estate Investment Trust
df_ftact.loc[(df_ftact.Taxpayer_Organizational_Type == 'TR'),'Taxpayer_Organizational_Name']='Texas Business Trust' #   TR – Texas Business Trust
# Record Type Code:
df_ftact.loc[(df_ftact.Record_Type_Code == 'U'),'Record_Type_Name']='Secretary of State (SOS) File Number' #   U = Secretary of State (SOS) File Number
df_ftact.loc[(df_ftact.Record_Type_Code == 'V'),'Record_Type_Name']='SOS Certificate of Authority (COA) File Number' #   V = SOS Certificate of Authority (COA) File Number
df_ftact.loc[(df_ftact.Record_Type_Code == 'X'),'Record_Type_Name']='Comptroller Assigned File Number' #   X = Comptroller Assigned File Number
df_ftact.head()

# (Description for context) SOS Charter/COA:
#   Depending on the Record Type Code value, this number
#   is the SOS, COA or Comptroller Assigned File Number.
#   If the Record Type Code is an 'X', this field will be
#   blank.  They do not have a current SOS Charter/COA.

,Taxpayer_Number,Taxpayer_Name,Taxpayer_Address,Taxpayer_City,Taxpayer_State,Taxpayer_Zip_Code,Taxpayer_County_Code,Taxpayer_Organizational_Type,Taxpayer_Phone_Number,Record_Type_Code,...,Secretary_of_State_File_Number,SOS_Charter_Date,SOS_Status_Date,SOS_Status_Code,Rigth_to_Tansact_Business_Code,Current_Exempt_Reason_Code,Exempt_Begin_Date,NAICS_Code,Taxpayer_Organizational_Name,Record_Type_Name
0,10004813456,MARSHALL E HAMILTON MD PA.,PO BOX 38206 C/O STEVE WINN CPA,DALLAS,TX,75238.0,57,AP,9726442142,X,...,NaN,NaN,NaN,NaN,A,NaN,NaN,621111.0,Texas Professional Association,Comptroller Assigned File Number
1,10058125732,CAP INVESTMENT GROUP,1801 HINKLE DR STE 150,DENTON,TX,76201.0,61,PB,9408914223,X,...,NaN,NaN,NaN,NaN,A,NaN,NaN,551112.0,Business General Partnership,Comptroller Assigned File Number
2,10082832105,"JDM RESOURCES, LTD",5105 BUCKLAND DR,MCKINNEY,TX,75070.0,43,PL,2145440124,X,...,NaN,NaN,NaN,NaN,A,NaN,NaN,525990.0,Texas Limited Partnership,Comptroller Assigned File Number
3,10100464006,CLARK INSURANCE,1945 CONGRESS ST STE A,PORTLAND,ME,4102.0,0,CF,2075232218,V,...,800878850.0,20071001.0,20071001.0,A,A,NaN,NaN,NaN,Foreign Profit,SOS Certificate of Authority (COA) File Number
4,10100798122,"FIASCHETTI ENTERPRISES, LP",5610 5TH ST,KATY,TX,77493.0,101,PL,8329717661,U,...,800242075.0,20030826.0,20090520.0,R,A,NaN,NaN,NaN,Texas Limited Partnership,Secretary of State (SOS) File Number


#### Responsibility_End_Reason_Name column

In [59]:
# Responsibility End Reason Code:
#   This is for mostly for Record Type Code value 'X'.
df_ftact.loc[(df_ftact.Responsibility_End_Reason_Code == 0),'Responsibility_End_Reason_Name']='Active or Inactive with no Reason Code' #     00 = Active or Inactive with no Reason Code
df_ftact.loc[(df_ftact.Responsibility_End_Reason_Code == 1),'Responsibility_End_Reason_Name']='Discountinued Doing Business' #     01 = Discountinued Doing Business
df_ftact.loc[(df_ftact.Responsibility_End_Reason_Code == 2),'Responsibility_End_Reason_Name']='Dissolved in Home State' #     02 = Dissolved in Home State
df_ftact.loc[(df_ftact.Responsibility_End_Reason_Code == 3),'Responsibility_End_Reason_Name']='Merged Out of Existence' #     03 = Merged Out of Existence
df_ftact.loc[(df_ftact.Responsibility_End_Reason_Code == 4),'Responsibility_End_Reason_Name']='Converted' #     04 = Converted
df_ftact.loc[(df_ftact.Responsibility_End_Reason_Code == 5),'Responsibility_End_Reason_Name']='Consolidated' #     05 = Consolidated
df_ftact.loc[(df_ftact.Responsibility_End_Reason_Code == 6),'Responsibility_End_Reason_Name']='Forfeited in Home State' #     06 = Forfeited in Home State
df_ftact.loc[(df_ftact.Responsibility_End_Reason_Code == 8),'Responsibility_End_Reason_Name']='No Nexus' #     08 = No Nexus
df_ftact.loc[(df_ftact.Responsibility_End_Reason_Code == 9),'Responsibility_End_Reason_Name']='No Nexus – Dates not the same' #     09 = No Nexus – Dates not the same
df_ftact.loc[(df_ftact.Responsibility_End_Reason_Code == 11),'Responsibility_End_Reason_Name']='Special Information Report' #     11 = Special Information Report
df_ftact.head()

,Taxpayer_Number,Taxpayer_Name,Taxpayer_Address,Taxpayer_City,Taxpayer_State,Taxpayer_Zip_Code,Taxpayer_County_Code,Taxpayer_Organizational_Type,Taxpayer_Phone_Number,Record_Type_Code,...,SOS_Charter_Date,SOS_Status_Date,SOS_Status_Code,Rigth_to_Tansact_Business_Code,Current_Exempt_Reason_Code,Exempt_Begin_Date,NAICS_Code,Taxpayer_Organizational_Name,Record_Type_Name,Responsibility_End_Reason_Name
0,10004813456,MARSHALL E HAMILTON MD PA.,PO BOX 38206 C/O STEVE WINN CPA,DALLAS,TX,75238.0,57,AP,9726442142,X,...,NaN,NaN,NaN,A,NaN,NaN,621111.0,Texas Professional Association,Comptroller Assigned File Number,Active or Inactive with no Reason Code
1,10058125732,CAP INVESTMENT GROUP,1801 HINKLE DR STE 150,DENTON,TX,76201.0,61,PB,9408914223,X,...,NaN,NaN,NaN,A,NaN,NaN,551112.0,Business General Partnership,Comptroller Assigned File Number,Active or Inactive with no Reason Code
2,10082832105,"JDM RESOURCES, LTD",5105 BUCKLAND DR,MCKINNEY,TX,75070.0,43,PL,2145440124,X,...,NaN,NaN,NaN,A,NaN,NaN,525990.0,Texas Limited Partnership,Comptroller Assigned File Number,Active or Inactive with no Reason Code
3,10100464006,CLARK INSURANCE,1945 CONGRESS ST STE A,PORTLAND,ME,4102.0,0,CF,2075232218,V,...,20071001.0,20071001.0,A,A,NaN,NaN,NaN,Foreign Profit,SOS Certificate of Authority (COA) File Number,Active or Inactive with no Reason Code
4,10100798122,"FIASCHETTI ENTERPRISES, LP",5610 5TH ST,KATY,TX,77493.0,101,PL,8329717661,U,...,20030826.0,20090520.0,R,A,NaN,NaN,NaN,Texas Limited Partnership,Secretary of State (SOS) File Number,Active or Inactive with no Reason Code


#### SOS_Status_Name Column

In [60]:
# (Context description) SOS Charter/COA:
#   Depending on the Record Type Code value, this number
#   is the SOS, COA or Comptroller Assigned File Number.
#   If the Record Type Code is an 'X', this field will be
#   blank.  They do not have a current SOS Charter/COA.

# SOS Status Code:
#   For Charter/COA Numbers:
df_ftact.loc[(df_ftact.SOS_Status_Code == 'A'),'SOS_Status_Name']='Active' #     A = Active
df_ftact.loc[(df_ftact.SOS_Status_Code == 'B'),'SOS_Status_Name']='Consolidated' #     B = Consolidated
df_ftact.loc[(df_ftact.SOS_Status_Code == 'C'),'SOS_Status_Name']='Converted' #     C = Converted
df_ftact.loc[(df_ftact.SOS_Status_Code == 'D'),'SOS_Status_Name']='Dissolved' #     D = Dissolved
df_ftact.loc[(df_ftact.SOS_Status_Code == 'E'),'SOS_Status_Name']='Expired' #     E = Expired
df_ftact.loc[(df_ftact.SOS_Status_Code == 'F'),'SOS_Status_Name']='Forfeited Franchise Tax' #     F = Forfeited Franchise Tax
df_ftact.loc[(df_ftact.SOS_Status_Code == 'G'),'SOS_Status_Name']='Miscellaneous' #     G = Miscellaneous
df_ftact.loc[(df_ftact.SOS_Status_Code == 'I'),'SOS_Status_Name']='Closed by FDIC' #     I = Closed by FDIC
df_ftact.loc[(df_ftact.SOS_Status_Code == 'J'),'SOS_Status_Name']='State Charter Pulled' #     J = State Charter Pulled
df_ftact.loc[(df_ftact.SOS_Status_Code == 'K'),'SOS_Status_Name']='Forfeited Registered Agent' #     K = Forfeited Registered Agent
df_ftact.loc[(df_ftact.SOS_Status_Code == 'L'),'SOS_Status_Name']='Forfeited Registered Office' #     L = Forfeited Registered Office
df_ftact.loc[(df_ftact.SOS_Status_Code == 'M'),'SOS_Status_Name']='Merger' #     M = Merger
df_ftact.loc[(df_ftact.SOS_Status_Code == 'N'),'SOS_Status_Name']='Forfeited Hot Check' #     N = Forfeited Hot Check
df_ftact.loc[(df_ftact.SOS_Status_Code == 'P'),'SOS_Status_Name']='Forfeited Court Order' #     P = Forfeited Court Order
df_ftact.loc[(df_ftact.SOS_Status_Code == 'R'),'SOS_Status_Name']='Reinstated' #     R = Reinstated
df_ftact.loc[(df_ftact.SOS_Status_Code == 'T'),'SOS_Status_Name']='Terminated' #     T = Terminated
df_ftact.loc[(df_ftact.SOS_Status_Code == 'W'),'SOS_Status_Name']='Withdrawn' #     W = Withdrawn
df_ftact.loc[(df_ftact.SOS_Status_Code == 'Y'),'SOS_Status_Name']='Dead at Conversion 69' #     Y = Dead at Conversion 69
df_ftact.loc[(df_ftact.SOS_Status_Code == 'Z'),'SOS_Status_Name']='Dead at Conversion 83' #     Z = Dead at Conversion 83
df_ftact.head()

,Taxpayer_Number,Taxpayer_Name,Taxpayer_Address,Taxpayer_City,Taxpayer_State,Taxpayer_Zip_Code,Taxpayer_County_Code,Taxpayer_Organizational_Type,Taxpayer_Phone_Number,Record_Type_Code,...,SOS_Status_Date,SOS_Status_Code,Rigth_to_Tansact_Business_Code,Current_Exempt_Reason_Code,Exempt_Begin_Date,NAICS_Code,Taxpayer_Organizational_Name,Record_Type_Name,Responsibility_End_Reason_Name,SOS_Status_Name
0,10004813456,MARSHALL E HAMILTON MD PA.,PO BOX 38206 C/O STEVE WINN CPA,DALLAS,TX,75238.0,57,AP,9726442142,X,...,NaN,NaN,A,NaN,NaN,621111.0,Texas Professional Association,Comptroller Assigned File Number,Active or Inactive with no Reason Code,NaN
1,10058125732,CAP INVESTMENT GROUP,1801 HINKLE DR STE 150,DENTON,TX,76201.0,61,PB,9408914223,X,...,NaN,NaN,A,NaN,NaN,551112.0,Business General Partnership,Comptroller Assigned File Number,Active or Inactive with no Reason Code,NaN
2,10082832105,"JDM RESOURCES, LTD",5105 BUCKLAND DR,MCKINNEY,TX,75070.0,43,PL,2145440124,X,...,NaN,NaN,A,NaN,NaN,525990.0,Texas Limited Partnership,Comptroller Assigned File Number,Active or Inactive with no Reason Code,NaN
3,10100464006,CLARK INSURANCE,1945 CONGRESS ST STE A,PORTLAND,ME,4102.0,0,CF,2075232218,V,...,20071001.0,A,A,NaN,NaN,NaN,Foreign Profit,SOS Certificate of Authority (COA) File Number,Active or Inactive with no Reason Code,Active
4,10100798122,"FIASCHETTI ENTERPRISES, LP",5610 5TH ST,KATY,TX,77493.0,101,PL,8329717661,U,...,20090520.0,R,A,NaN,NaN,NaN,Texas Limited Partnership,Secretary of State (SOS) File Number,Active or Inactive with no Reason Code,Reinstated


#### Rigth_to_Tansact_Business_Name Column

In [61]:
# Exempt Reason Code:
#   blank = Not Exempt
#   rest = Exempt for various reasons.  A list of value descriptions
#          may be requested separately.

# Right to Transact Business Code:
#   blank = Franchise Tax Ended
df_ftact.loc[(df_ftact.Rigth_to_Tansact_Business_Code == 'A'),'Rigth_to_Tansact_Business_Name']='Active' #   A = Active
df_ftact.loc[(df_ftact.Rigth_to_Tansact_Business_Code == 'D'),'Rigth_to_Tansact_Business_Name']='Active – Eligible for Termination/Withdrawl' #   D = Active – Eligible for Termination/Withdrawl
df_ftact.loc[(df_ftact.Rigth_to_Tansact_Business_Code == 'N'),'Rigth_to_Tansact_Business_Name']='Forfeited' #   N = Forfeited
df_ftact.loc[(df_ftact.Rigth_to_Tansact_Business_Code == 'I'),'Rigth_to_Tansact_Business_Name']='Franchise Tax Involuntarily Ended' #   I = Franchise Tax Involuntarily Ended
df_ftact.loc[(df_ftact.Rigth_to_Tansact_Business_Code == 'U'),'Rigth_to_Tansact_Business_Name']='Franchise Tax Not Established' #   U = Franchise Tax Not Established
df_ftact.head()

,Taxpayer_Number,Taxpayer_Name,Taxpayer_Address,Taxpayer_City,Taxpayer_State,Taxpayer_Zip_Code,Taxpayer_County_Code,Taxpayer_Organizational_Type,Taxpayer_Phone_Number,Record_Type_Code,...,SOS_Status_Code,Rigth_to_Tansact_Business_Code,Current_Exempt_Reason_Code,Exempt_Begin_Date,NAICS_Code,Taxpayer_Organizational_Name,Record_Type_Name,Responsibility_End_Reason_Name,SOS_Status_Name,Rigth_to_Tansact_Business_Name
0,10004813456,MARSHALL E HAMILTON MD PA.,PO BOX 38206 C/O STEVE WINN CPA,DALLAS,TX,75238.0,57,AP,9726442142,X,...,NaN,A,NaN,NaN,621111.0,Texas Professional Association,Comptroller Assigned File Number,Active or Inactive with no Reason Code,NaN,Active
1,10058125732,CAP INVESTMENT GROUP,1801 HINKLE DR STE 150,DENTON,TX,76201.0,61,PB,9408914223,X,...,NaN,A,NaN,NaN,551112.0,Business General Partnership,Comptroller Assigned File Number,Active or Inactive with no Reason Code,NaN,Active
2,10082832105,"JDM RESOURCES, LTD",5105 BUCKLAND DR,MCKINNEY,TX,75070.0,43,PL,2145440124,X,...,NaN,A,NaN,NaN,525990.0,Texas Limited Partnership,Comptroller Assigned File Number,Active or Inactive with no Reason Code,NaN,Active
3,10100464006,CLARK INSURANCE,1945 CONGRESS ST STE A,PORTLAND,ME,4102.0,0,CF,2075232218,V,...,A,A,NaN,NaN,NaN,Foreign Profit,SOS Certificate of Authority (COA) File Number,Active or Inactive with no Reason Code,Active,Active
4,10100798122,"FIASCHETTI ENTERPRISES, LP",5610 5TH ST,KATY,TX,77493.0,101,PL,8329717661,U,...,R,A,NaN,NaN,NaN,Texas Limited Partnership,Secretary of State (SOS) File Number,Active or Inactive with no Reason Code,Reinstated,Active


#### Formating data
* changing float to int
* adding datetime format

In [62]:
df_ftact['Taxpayer_Zip_Code'] = df_ftact['Taxpayer_Zip_Code'].fillna(0)
df_ftact['SOS_Charter_Date'] = df_ftact['SOS_Charter_Date'].fillna(0)
df_ftact['SOS_Status_Date'] = df_ftact['SOS_Status_Date'].fillna(0)
df_ftact['Secretary_of_State_File_Number'] = df_ftact['Secretary_of_State_File_Number'].fillna(0)
df_ftact['NAICS_Code'] = df_ftact['NAICS_Code'].fillna(0)
df_ftact['Current_Exempt_Reason_Code'] = df_ftact['Current_Exempt_Reason_Code'].fillna(0)

df_ftact['Taxpayer_Zip_Code'] = df_ftact['Taxpayer_Zip_Code'].astype(np.int64)
df_ftact['SOS_Charter_Date'] = df_ftact['SOS_Charter_Date'].astype(np.int64)
df_ftact['SOS_Status_Date'] = df_ftact['SOS_Status_Date'].astype(np.int64)
df_ftact['Responsibility_Beginning_Date'] = df_ftact['Responsibility_Beginning_Date'].astype(np.int64)
df_ftact['Secretary_of_State_File_Number'] = df_ftact['Secretary_of_State_File_Number'].astype(np.int64)
df_ftact['NAICS_Code'] = df_ftact['NAICS_Code'].astype(np.int64)
df_ftact['Current_Exempt_Reason_Code'] = df_ftact['Current_Exempt_Reason_Code'].astype(np.int64)

df_ftact['SOS_Charter_Date'] = pd.to_datetime(df_ftact["SOS_Charter_Date"], format='%Y%m%d', errors='coerce')
df_ftact['SOS_Status_Date'] = pd.to_datetime(df_ftact["SOS_Status_Date"], format='%Y%m%d', errors='coerce')
df_ftact['Responsibility_Beginning_Date'] = pd.to_datetime(df_ftact["Responsibility_Beginning_Date"], format='%Y%m%d', errors='coerce')

df_ftact['SOS_Charter_Date'] = df_ftact['SOS_Charter_Date'].dt.normalize()
df_ftact['SOS_Status_Date'] = df_ftact['SOS_Status_Date'].dt.normalize()
df_ftact['Responsibility_Beginning_Date'] = df_ftact['Responsibility_Beginning_Date'].dt.normalize()

df_ftact = df_ftact[df_ftact['Taxpayer_Zip_Code']!=0]
df_ftact.head()

,Taxpayer_Number,Taxpayer_Name,Taxpayer_Address,Taxpayer_City,Taxpayer_State,Taxpayer_Zip_Code,Taxpayer_County_Code,Taxpayer_Organizational_Type,Taxpayer_Phone_Number,Record_Type_Code,...,SOS_Status_Code,Rigth_to_Tansact_Business_Code,Current_Exempt_Reason_Code,Exempt_Begin_Date,NAICS_Code,Taxpayer_Organizational_Name,Record_Type_Name,Responsibility_End_Reason_Name,SOS_Status_Name,Rigth_to_Tansact_Business_Name
0,10004813456,MARSHALL E HAMILTON MD PA.,PO BOX 38206 C/O STEVE WINN CPA,DALLAS,TX,75238,57,AP,9726442142,X,...,NaN,A,0,NaN,621111,Texas Professional Association,Comptroller Assigned File Number,Active or Inactive with no Reason Code,NaN,Active
1,10058125732,CAP INVESTMENT GROUP,1801 HINKLE DR STE 150,DENTON,TX,76201,61,PB,9408914223,X,...,NaN,A,0,NaN,551112,Business General Partnership,Comptroller Assigned File Number,Active or Inactive with no Reason Code,NaN,Active
2,10082832105,"JDM RESOURCES, LTD",5105 BUCKLAND DR,MCKINNEY,TX,75070,43,PL,2145440124,X,...,NaN,A,0,NaN,525990,Texas Limited Partnership,Comptroller Assigned File Number,Active or Inactive with no Reason Code,NaN,Active
3,10100464006,CLARK INSURANCE,1945 CONGRESS ST STE A,PORTLAND,ME,4102,0,CF,2075232218,V,...,A,A,0,NaN,0,Foreign Profit,SOS Certificate of Authority (COA) File Number,Active or Inactive with no Reason Code,Active,Active
4,10100798122,"FIASCHETTI ENTERPRISES, LP",5610 5TH ST,KATY,TX,77493,101,PL,8329717661,U,...,R,A,0,NaN,0,Texas Limited Partnership,Secretary of State (SOS) File Number,Active or Inactive with no Reason Code,Reinstated,Active


## Upload DF's to Database
* Adding database connection
* Defining the Engine
** I was getting charmap error when attempting to drop the data to the database. I defined encoding = utf-8, yet it still did not work. Only when I hardcoded charset within the engine string is when the error finally went away.

In [63]:
connection_string = f"{mysql_user}:{mysql_password}@localhost:3306/{db_name}?charset=utf8"
engine = create_engine(f'mysql://{connection_string}')

In [64]:
engine.table_names()

['ccim',
 'franchise_tax_info',
 'franchise_tax_info_date',
 'ftoffdir',
 'sales_tax_info',
 'sales_tax_info_date']

#### Creating two variables for today's date and today's datetime

In [65]:
currentDT = datetime.datetime.now()
DateTimeSent = currentDT.strftime("%Y-%m-%d %H:%M:%S")
dateCSV = currentDT.strftime("%Y-%m-%d")
print(dateCSV)
print(DateTimeSent)

2020-03-25
2020-03-25 02:13:35


## Calling Database tables for crosreferencing df data, to have non-duplicated data
* Grabing data from the database and storing the tax number column into a dataframe

In [66]:
ftact_in_db = pdsql.read_sql("SELECT Taxpayer_Number FROM franchise_tax_info",engine)

print(f"Data count for ftact from the database : {len(ftact_in_db)}\n)

try:
    if df_fran.size != 0:
        print(f"\nData count from the new df data for df_fran: {len(df_fran)}")
except Exception as e:
    print("df_fran does not exist. Check your data source if it is available")
    
try:
    if df_ftact.size != 0:
        print(f"Data count from the new df data for df_ftact: {len(df_ftact)}")
except Exception as e:
    print("df_ftact does not exist. Check your data source if it is available")    

Data count for ftact from the database : 0
Data count for stact from the Database: 0
Data count for ftoffdir from the Database: 0
Data count from the new df data for df_ftact: 4236082
Data count from the new df data for df_stact: 1554488
df_ftoffdir does not exist. Check your data source if it is available


## FTACT aka df_ftact
#### Checking table df with df data to make sure their are not duplicate tax paying numbers
* filtering new ftact with data from the database
* Checking data for ftact and also adding a new column of today's date and time
* Appending new companies (df_ftact) to  csv and Database

In [67]:
try:
    df_ftact = df_ftact[~df_ftact['Taxpayer_Number'].astype(int).isin(ftact_in_db['Taxpayer_Number'].astype(int))]
    if df_ftact.size != 0:
        df_ftact['DateTime'] = DateTimeSent
        print(f"There are {len(ftact_in_db)} data attributes in ftact table from the database\n{len(df_ftact)} new companies, based on tax payer number from filtered data df_tact")
        df_ftact.to_sql(name='franchise_tax_info', con=engine, if_exists='append', index=False, chunksize=1000)
        print(f"ftact to database append, completed")
        f = open('HotelOccupancyTaxData/formattedData/DBUploadRecord.txt','a+')
        f.write(f'{DateTimeSent}\nftact_{dateCSV}.csv, {len(df_ftact)}, franchise_tax_info table, {len(ftact_in_db)}\n')
        f.close()
    else:
        print("No new data")
        f = open('HotelOccupancyTaxData/formattedData/DBUploadRecord.txt','a+')
        f.write(f'{DateTimeSent}\nftact_{dateCSV}.csv, {len(df_ftact)}, franchise_tax_info table, {len(ftact_in_db)}\n')
        f.close()
except Exception as e:
    print(f"Something went wrong, df_ftact was not able to append to database or no new data: {e}")

There are 0 data attributes in ftact table from the database
4236082 new companies, based on tax payer number from filtered data df_tact
ftact to database append, completed


## Call the tables within the database and store into a variable
* Going to compare new data from database with the df_fran

In [234]:
ftact_date_in_db = pdsql.read_sql("SELECT Taxpayer_Number FROM franchise_tax_info_date",engine)
print(f"There are {len(ftact_date_in_db)} records in frachise tax permit date table.\n")

There are 0 records in frachise tax permit date table.
There are 0 records in sales tax permit date table. 


## fran aka df_fran
#### Checking table df with df data to make sure their are not duplicate tax paying numbers
* filtering new df_fran with data from the database
* Checking data for df_fran and also adding a new column of today's date and time
* Appending new companies (fran) to  csv and Database

In [235]:
try:
    df_fran = df_fran[~df_fran['Taxpayer_Number'].astype(int).isin(ftact_date_in_db['Taxpayer_Number'].astype(int))]
    if df_fran.size != 0:
        df_fran['DateTime'] = DateTimeSent
        print(f"There are {len(ftact_date_in_db)} data attributes in df_fran table from the database\n{len(df_fran)} new companies, based on tax payer number from filtered data df_fran")
        df_fran.to_sql(name='franchise_tax_info_date', con=engine, if_exists='append', index=False, chunksize=1000)
        print(f"df_fran to database append, completed")
        f = open('HotelOccupancyTaxData/formattedData/DBUploadRecord.txt','a+')
        f.write(f'fran_{dateCSV}.csv, {len(df_fran)}, franchise_tax_info_date table, {len(ftact_date_in_db)}\n')
        f.close()
    else:
        print("No new data")
        f = open('HotelOccupancyTaxData/formattedData/DBUploadRecord.txt','a+')
        f.write(f'fran_{dateCSV}.csv, {len(df_fran)}, franchise_tax_info_date table, {len(ftact_date_in_db)}\n')
        f.close()
except Exception as e:
    print(f"Something went wrong, df_fran was not able to append to database: {e}")

There are 0 data attributes in df_fran table from the database
111888 new companies, based on tax payer number from filtered data df_fran
df_fran to database append, completed


<!-- #### Appending new companies (fran) to  csv and Database -->